# 第11章：接口——从协议到抽象基类
> 抽象类表示接口。

> 非常不建议编写抽象基类(强接口)，因为很容易过度设计。除非你在编写框架。

> 如果你告诉解释器一个对象是鸭子，解释器就尽最大全力把它当作鸭子去对待。

## 使用isinstance而不是type is
isinstance在检查对象是否是类及类的超类的实例，远比type(obj) is CLASS更有利于继承。  
看下面一个例子，我们编写一个聪明的人，他会丢掉垃圾，吃掉食物，把硬币存起来。

In [2]:
class Rubbish:
    pass

class Food:
    pass

class Coin:
    pass

class Person:
    def handle(self, obj):
        if type(obj) is Rubbish:
           self.throw(obj)
        elif type(obj) is Food:
            self.eat(obj)
        elif type(obj) is Coin:
            self.save(obj)
        else:
            print(f"I don't know hot to handle type {type(obj).__name__}")

    def eat(self, obj):
        print(f"eating {obj}...")

    def throw(self, obj):
        print(f"throwing {obj}...")

    def save(self, obj):
        print(f"saving {obj}...")
        

它工作得很好。但如果我们要添加对布丁（一种食物）的处理呢？  
Pudding会继承Food，但是Person此时不知道应该如何处理Pudding。因为type(obj) is not Food.  
使用isinstance就没有问题了。  

In [5]:
class Rubbish:
    pass

class Food:
    pass

class Coin:
    pass

class Person:
    def handle(self, obj):
        if isinstance(obj, Rubbish):
           self.throw(obj)
        elif isinstance(obj, Food):
            self.eat(obj)
        elif isinstance(obj, Coin):
            self.save(obj)
        else:
            print(f"I don't know hot to handle type {type(obj).__name__}")

    def eat(self, obj):
        print(f"eating {obj}...")

    def throw(self, obj):
        print(f"throwing {obj}...")

    def save(self, obj):
        print(f"saving {obj}...")
        
class Pudding(Food):
    pass

pudding = Pudding()
person = Person()
person.handle(pudding)

eating <__main__.Pudding object at 0x7f1f16d79070>...


> 顺带一提，这里常用single-dispatch解耦。此前pangolin项目中从openfst出来的文本读法类型，也应该用value-dispatch解耦的。但是没有采用，是出于什么考虑呢...

## 将大部分异常交给解释器，并立即失败
在工程开发中，并不是所有的异常都要自己处理的。  
一个好的原则是：
- 如果输入来自用户：应该手动处理一切异常，确保对用户的每个输入都有响应。模块能通过猴子测试。
- 如果在模块内部，确保debug时抛出的异常信息能让开发者，或后来的维护者意识到发生了什么即可。这样的错误不应该导致维护者需要修改模块，而应该是该维护者的外部调用不当。

编写一个处理jpg文件的脚本。  
提供给开发者:

In [13]:
import os

def handle_jpg(dir_path, func):
    files = os.listdir(dir_path)
    for file_name in files:
        file_path = os.path.join(dir_path, file_name)
        if file_path.endswith(".jpg"):
            func(file_path)
    print("handle_jpg done.")
# 这里故意触发几种异常。
# 以说明解释器抛出的异常在大多数时候足以令开发者明白发生了什么。
try:
    handle_jpg("my jpg folder", lambda x: print(x))
except Exception as e:
    print(e)
    
try:
    handle_jpg("./", lambda x,y: print(x,y))
except Exception as e:
    print(e)

try:
    handle_jpg("./", "my function?")
except Exception as e:
    print(e)

[Errno 2] No such file or directory: 'my jpg folder'
<lambda>() missing 1 required positional argument: 'y'
'str' object is not callable


但如果该模块是提供给用户的，就必须非常清晰:

In [ ]:
import os
from types import FunctionType

def handle_jpg(dir_path, func):
    if not os.path.isdir(dir_path):
        print("给出的dir_path不是一个存在的合法路径，请检查.")
        return False
    if not isinstance(func, FunctionType):
        print(f"func需要是一个接受图片路径作为参数的函数.而不是{type(func).__name__}类型")
        return False
    files = os.listdir(dir_path)
    jpg_paths = [os.path.join(dir_path, file_name) for file_name in files if file_name.endswith(".jpg")]
    # 如果没有jpg，最好还要发出警告
    if len(jpg_paths)==0:
        print(f"Warning: No jpg find in {dir_path}.")
        return True
    # 执行用户的函数，我们无法预期该函数会发生什么，故尽最大可能捕获。
    for jpg in jpg_paths:
        try:
            func(jpg)
        except Exception as e:
            print(f"使用输入func处理{jpg}时发生错误，错误信息:\n{e}")
            return False
    print("handle_jpg done.")
    return True

在添加比较完备的异常处理后，一个7行的函数甚至可以拓展到23行。  
这使得阅读核心代码变得困难。因此，绝不要在非用户接口处定义过多的异常捕获及说明函数。而应该将这一工作交给解释器。  
即使在写业务，也应该实现用户交互逻辑与核心算法的分离。  

## 继承抽象类
在python中，即使是抽象类，也不强制要求实现所有的方法。只有抽象类中的抽象方法，是继承该抽象类所必须实现的。

In [18]:
from collections.abc import MutableSequence

class Test(MutableSequence):
    def __setitem__(self, position, value):
        print(f"set position {position} to {value}")
    
    def __delitem__(self, position):
        print(f"delete position {position} obj.")

    def __getitem__(self, position):
        print(f"get item from position {position}")

    def __len__(self) -> int:
        return 10

    def insert(self, position, value):
        print(f"insert {value} to {position}")
# 以上的方法都是抽象方法，因而必须实现，否则会在尝试实例化对象时抛出异常.
# Can't instantiate abstract class Test with abstract method [abc method list]
test = Test()

## TODO: 11.6 标准库中的抽象基类